In [ ]:
# path to big file
path = 'data/da.txt'

# its 45 gb file, so i will not load it in memory

# we will read it line by line

# we will create a generator function to read it line by line
def read_file(path):
    for line in open(path):
        yield line

    close(path)

# now we will use this generator function to read it line by line
# porint 5 lines

entries = {
    0 : ''
}
counter = 0
for line in read_file(path):
    # entries are split by \n\n
    if line == '\n':
        # cut at 2000 characters
        lim = 500
        if len(entries[counter]) > lim:
            entries[counter] = entries[counter][:lim]

            # end at final ., !, ? or \n

            # find last occurence of ., !, ? or \n
            last_occurence = 0
            for i in range(len(entries[counter])):
                if entries[counter][i] in ['.', '!', '?', '\n']:
                    last_occurence = i

            # cut at last occurence
            entries[counter] = entries[counter][:last_occurence]
            

        counter += 1
        if counter > 10000:
            break    

        entries[counter] = ''
    else:
        entries[counter] += line

    
entries


In [ ]:
sporgsmål = {
    0 : 'Fortæl om emnet',
    # 1 : 'Hvad er problemet?',
    # 2 : 'Hvad er løsningen?',
    # 3 : 'Hvem ville være interesseret i?',
    4 : 'Uddyb om emnet',
    5 : 'Forestil dig at du har en brevkasse, giv et svar på emnet',
    6 : 'Du er politiker og bliver interviewet om emnet, hvad fortæller du?',
    7 : 'Du er journalist og skal skrive en artikel om emnet, hvad skriver du?',
    8 : 'Du bliver interviewet om emnet',
}

sporgsmål = {
    # "Giv råd" : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forestil dig, at du skal komme med råd ud fra emnet.  Hvis der ikke er nok at snakke om, så find på noget mere. Ignorer smiley:",
    # "Politiker; interview" : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forestil dig, at du er politiker og skal tale ud fra emnet, men nævn ikke, at du er politiker.  Afslut ikke 'med venlig hilsen', og hvis der ikke er nok at snakke om, så find på noget. Ignorer smiley:",
    # "Journalist; artikel" : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forestil dig, at du er journalist og skal skrive en artikel ud fra emnet.  Hvis der ikke er nok at snakke om, så find på noget mere. Udelad titel. Ignorer smiley:",
    # "At blive spurgt og give sin mening" : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forestil dig, at du bliver spurgt om emnet og skal give din mening.  Hvis der ikke er nok at snakke om, så find på noget mere. Ignorer smiley: ",
    # "Uddyb og forklar " : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forklar og uddyb emnet.  Hvis der ikke er nok at snakke om, så find på noget mere. Ignorer smiley: ",
    # "Debatforum" : "Skriv om følgende på dansk - vælg en tone og stil, som du synes er passende.  Forestil dig, at du er inde på et uformelt debatforum og skal skrive en kommentar.  Hvis der ikke er nok at snakke om, så find på noget mere. Ignorer smiley: "
    'Fortæl om emnet' : 'forklar og uddyb emnet',
}

In [ ]:
# get openai to write about the same topics
from openai import OpenAI
import os
import dotenv
import numpy as np
import concurrent.futures

import time

# use wrap text
import textwrap

def wrap_text(text, width=80):
    return "\n".join(textwrap.wrap(text, width=width))

model = "gpt-3.5-turbo-0613"

#load .env file
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def save(model, article_path, question, response, path="data/from_big_data_file_ai/"):

    # save to file
    out_path = os.path.join(path,model, os.path.basename(article_path))
    print(out_path)
    
    # save as title\n model\n text
    with open(out_path, "w") as f:
        f.write(question + "\n")
        f.write(model + "\n")
        f.write(response)

        
def get_ai_article(sprg, text, idx, sprg_idx, model=model):
        # print(model)
        # print(sprg)
        # print(text)


    # try:
        completion = client.chat.completions.create(
        model=model,
        messages=[
            # {"role": "system", "content": "Du vil få et stykke text og hertil et spørgsmål. Svar altid på dansk. Formater ikke med 'emnet: ...', 'svar: ...', 'spørgsmål: ...', eller lignende. Skriv naturligt.  Du bør ikke være usikker, 'take full creative liberty' Husk at påtage dig rollen som du svarer som."},
            # {"role": "user", "content": f"Spørgsmål: {sprgs} \n\n Text: {text}"},
            {
                "role": "system", 
                "content" : sprg,

            },
            {"role": "user", "content": text,}
        ]
        )

        # return completion.choices[0].message.content
        save(model = model,
                article_path = f"/{idx}_{sprg_idx}.txt",
                question = sprg,
                response = completion.choices[0].message.content,
                path="data/from_big_data_file_ai/")
        
        # save article
        save(model = '',
                article_path = f"/{idx}.txt",
                question = sprg,
                response = text,
                path="data/from_big_data_file/")
    
    # except Exception as e:
    #     print(e)
    #     print("Rate limit reached, sleeping for 30 milliseconds")
    #     time.sleep(0.3)
    #     return get_ai_article(sprgs, text, model)  # Retry the request

    

def predict_parallel(sprgs, texts, idxes, questions_idxes):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(get_ai_article, sprgs, texts, idxes, questions_idxes))
    return results


entries_not_processed = []
idxes = []
for idx in entries:

    # check if entry already exists
    if os.path.exists(f"data/from_big_data_file/{idx}.txt"):
        pass
    else:
        entries_not_processed.append(entries[idx])
        idxes.append(idx)
    


# get questions
questions_idxes = np.random.choice(np.arange(len(sporgsmål)), size=len(entries_not_processed), replace=True)
questions = [sporgsmål[list(sporgsmål.keys())[idx]] for idx in questions_idxes]
len(questions), len(entries_not_processed)
    
# get ai to write about the same topics
results = predict_parallel(questions, entries_not_processed, idxes, questions_idxes)
        
